## Payment Priority - Insights from Equifax Analytics Dataset

### Author: Barry Zihao Qie

In [ ]:
'''
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
'''

In [ ]:
import os

path=os.getcwd()

print(path)

In [ ]:
os.chdir('C:\\Users\\zihao\\OneDrive\\Desktop\\Citi Work\\ABS Research\\1010 XMLquery\\Equifax Analytics Dataset')

path =os.getcwd()

print(path)

In [ ]:
# import statements
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.dates as mdates
import seaborn as sns

import py1010
from dateutil.relativedelta import relativedelta, FR

In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [ ]:
cur_month= datetime.now().replace(day=1).strftime(format="%#m/%Y")
prev_month = (datetime.now().replace(day=1) + relativedelta(months=-1)).strftime(format="%#m/%Y")
print(cur_month)
print(prev_month)

In [ ]:
gateway  = "https://www2.1010data.com/cgi-bin/gw"
tablename = "pub.fin.efxas.client.v3.v3_panel_data"
username = "citi_zqie"
password = "Barry0228Qzh"

In [ ]:
folder_path='C:\\Users\\zihao\\OneDrive\\Desktop\\Citi Work\\ABS Research\\1010 XMLquery\\Equifax Analytics Dataset'
print(folder_path)

In [ ]:
query_path = folder_path
files=os.listdir(query_path)
query_list=[]
for i in files:
    query_list.append(i)

print(len(query_list))
print(query_list)

In [ ]:
from argparse import Namespace

settings = Namespace(
    
    #Chart Settings
    title_fontsize = 18,
    fig_size=(12,6),
    
    # BackUP Settings
    root_directory = 'C:\\Users\\zihao\\OneDrive\\Desktop\\Citi Work\\ABS Research\\Equifax Dataset Charts',
    report_directory = 'Chart_Content',
    backup_directory = 'BACKUP',
    
    #Chart Name Settings
    report_name = 'ExampleReport',
    rep_extn = ".png",
    backup_name_format = '%Y-%m-%d'
)

In [ ]:
start_date = input('What is the start date of your plot Specify to this format: 2005-07-01')

In [ ]:
print(start_date)
type(start_date)

In [ ]:
def Get_1010Data(q):
    session = py1010.Session(gateway,username,password,py1010.KILL)
    tablename = "pub.fin.efxas.client.v3.v3_panel_data"
    
    row_list = []
    headers = []
    try:
        query = session.query(tablename, q)
        query.run()
        for r in range(query.nrows):
            row_list.append(query.getrow(r))
        for c in query.cols:
            headers.append(c.name)
        
        df = pd.DataFrame(row_list,columns=headers)
        for col in df.columns:
            if isinstance(df[col][1],bytes):
                df[col] = df[col].str.decode("utf-8")
    
    except py1010.TentenException as e:
        print("Error: {}".format(e))
    
    session.close()
    print("result saved as a dataframe, and 1010 session closed")
    
    return df

In [ ]:
def Get_firsttoroll_probab(masterpivot,fico_bucket):
    firstroll_1catDQ = masterpivot.xs(fico_bucket,level='fico_bucket').xs(1,level="rollProdTypes2")
    firstroll_1catDQ['consumer_count_1catDQ'] = firstroll_1catDQ.groupby(level=['date'])['unique_borrower_count'].transform('sum')
    firstroll_1catDQ['firstroll_probab']='blank'
    
    for i in range(0,len(firstroll_1catDQ)):
        firstroll_1catDQ['firstroll_probab'].iloc[i] = (firstroll_1catDQ['unique_borrower_count'].iloc[i]/firstroll_1catDQ['consumer_count_1catDQ'].iloc[i])*100
        
    firstroll_1catDQ=firstroll_1catDQ.reset_index(level='date')
    
    return firstroll_1catDQ

In [ ]:
def Get_Lasttoremain_probab(masterpivot,fico_bucket):
    LastCurr_3catDQ = masterpivot.xs(fico_bucket,level='fico_bucket').xs(3,level="rollProdTypes2")
    LastCurr_3catDQ['consumer_count_3catDQ'] = LastCurr_3catDQ.groupby(level=['date'])['unique_borrower_count'].transform('sum')
    LastCurr_3catDQ['Lastroll_probab']='blank'
    
    for i in range(0,len(LastCurr_3catDQ)):
       LastCurr_3catDQ['Lastroll_probab'].iloc[i] = (LastCurr_3catDQ['unique_borrower_count'].iloc[i]/LastCurr_3catDQ['consumer_count_3catDQ'].iloc[i])*100
                
    LastCurr_3catDQ=LastCurr_3catDQ.reset_index(level='date')
    
    return LastCurr_3catDQ

In [ ]:
def plot_metric_full_hist(metric_df,start_date,y_label,plottitle,*args):
    
    fig = plt.figure(figsize=(12,6))
    ax = fig.add_subplot(1,1,1)
    c = ['#002060','#00BDF2','#99ABC7','#CCF2FC','99E2E3','#00B0B9','#FFD700','#A9561E','#E6DAA6']
    
    for i in range(0,len(args)):
        ax.plot(metric_df.loc[metric_df['date']>=start_date].xs(args[i],level="fico_bucket")['date'], metric_df.loc[metric_df['date']>=start_date].xs(args[i],level="fico_bucket").iloc[:,-1],c=c[i],ls='-',label=args[i],fillstyle='none',linewidth = 4.0)
    
    ax.set_xticklabels([str(i) for i in metric_df['date'].values], fontsize=15)
    ax.set_yticklabels([str(i) for i in ax.get_yticks()],fontsize=15)
    
    ax.tick_params(axis='x',colors='black')
    ax.tick_params(axis='y', colors='black')
    
    plt.xticks(rotation= 30)
    plt.ylabel(y_label,fontsize = 16)
    
    plt.legend(loc = "upper center", ncol = len(args), prop = {'size':16}, bbox_to_anchor = (0.5,1.1), fancybox =True, frameon = False)
    
    ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(6)))
                               
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%y"))
    ax.xaxis.set_minor_formatter(mdates.DateFormatter("%b-%y"))
                                 
    ax.grid(axis='y',linestyle='--',dashes=(5,5))
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
                                 
    ax.yaxis.set_ticks_position('none')
    plt.gca().spines['bottom'].set_color('gray')

    plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,,.0f}'))
    fig.suptitle(plottitle,fontsize = 18)

    plt.show()
    
    return fig

In [ ]:
def save_chart(fig, figname):
    
    rep_path = os.path.join(settings.root_directory, settings.report_directory)
    if not os.path.exists(rep_path):
        os.makedirs(rep_path)
    
    filename = figname + settings.rep_extn
    fig.savefig(os.path.join(rep_path,filename))
    
    backup_path=os.path.join(rep_path, settings.backup_directory)     
    if not os.path.exists(backup_path):
        os.makedirs(backup_path)
    
    backup_date = datetime.today().strftime(settings.backup_name_format)
    backup_filename = figname + "_" + backup_date + settings.rep_extn
    fig.savefig(os.path.join(backup_path, backup_filename))
    
    print('chart saved to the target and BACKUP directory in this following directory C:\\Users\\zihao\\Desktop\\Citi Work\\ABS Research')

### ___Main___

### Payment Priority starts here

In [ ]:
file=open('Payment Priority - Roll and Save.txt','r')
lines=file.readlines()
q = [line.rstrip().lstrip() for line in lines]
file.close()
q="".join(q)

print(q)

In [ ]:
payment_priority_raw = Get_1010Data(q)

#average runtime: 20min

In [ ]:
payment_priority_raw1 = payment_priority_raw 
# save a copy of the data and work with raw1 later

In [ ]:
len(payment_priority_raw1)

In [ ]:
pd.concat([payment_priority_raw1.head(15),payment_priority_raw1.tail(15)])

In [ ]:
payment_priority_raw1 = payment_priority_raw1.set_index(['date','fico_bucket','rollProdTypes2'])

In [ ]:
payment_priority_pivot = pd.pivot_table(payment_priority_raw1,values=['unique_borrower_count'], columns=['date','fico_bucket','rollProdTypes2','rollAuto','rollFM','rollBC','rollPL'],aggfunc=np.sum).transpose()

payment_priority_pivot.head(15)

### Approach #1 First to Roll

### Background Statistics: FICO Distribution for borrowers that only have 1 type of loan within a 6-m rolling window

In [ ]:
roll90p_1cat_pivot.index

In [ ]:
roll90p_1cat_pivot=payment_priority_pivot.xs(1,level="rollProdTypes2").groupby(level=[0,'date','fico_bucket'])['unique_borrower_count'].sum()
#roll90p_1cat_pivot = roll90p_1cat_pivot.reset_index(level=0)

In [ ]:
#roll90p_1cat_pivot=roll90p_1cat_pivot.drop(columns=['date'])

In [ ]:
#roll90p_1cat_pivot.head()

In [ ]:
#roll90p_1cat_pivot = roll90p_1cat_pivot.reset_index(level='date')

In [ ]:
roll90p_1cat_pivot['date']=pd.to_datetime(roll90p_1cat_pivot['date'], infer_datetime_format=True)

In [ ]:
roll90p_1cat_pivot.head(10)

In [ ]:
save_chart(
    plot_metric_full_hist(roll90p_1cat_pivot,start_date,'# of Unique Borrower','FICO Distribution for Borrowers with Roll to 90+ DQ in only 1 Category','below 660','660-720','above 720'),
    'fico dist_1cat'
)

#### The loan category being the first to roll into serious DQ, for various FICO bands

In [ ]:
subprime_firsttoroll = Get_firsttoroll_probab(payment_priority_pivot,'below 660')
nearprime_firsttoroll = Get_firsttoroll_probab(payment_priority_pivot,'660-720')
prime_firsttoroll = Get_firsttoroll_probab(payment_priority_pivot,'above 720')

In [ ]:
subprime_firsttoroll['date']=pd.to_datetime(subprime_firsttoroll['date'], infer_datetime_format=True)
nearprime_firsttoroll['date']=pd.to_datetime(nearprime_firsttoroll['date'], infer_datetime_format=True)
prime_firsttoroll['date']=pd.to_datetime(prime_firsttoroll['date'], infer_datetime_format=True)

### Subprime Priority of Payment

In [ ]:
fig1=plt.figure(figsize=(12,6))
ax=fig1.add_subplot(1,1,1)

c=['#002060','#00BDF2','#99ABC7','#CCF2FC','99E2E3','#00B0B9','#FFD700','#A9561E','#E6DAA6']

ax.plot(subprime_firsttoroll.loc[subprime_firsttoroll['date']>=start_date].xs(1,level="rollAuto")['date'],subprime_firsttoroll.loc[subprime_firsttoroll['date']>=start_date].xs(1,level="rollAuto")['firstroll_probab'], c=c[0],ls='-',label='Auto',fillstyle='none',linewidth = 4.0)
ax.plot(subprime_firsttoroll.loc[subprime_firsttoroll['date']>=start_date].xs(1,level="rollPL")['date'],subprime_firsttoroll.loc[subprime_firsttoroll['date']>=start_date].xs(1,level="rollPL")['firstroll_probab'], c=c[1],ls='-',label='Personal Loan',fillstyle='none',linewidth = 4.0)
ax.plot(subprime_firsttoroll.loc[subprime_firsttoroll['date']>=start_date].xs(1,level="rollBC")['date'],subprime_firsttoroll.loc[subprime_firsttoroll['date']>=start_date].xs(1,level="rollBC")['firstroll_probab'], c=c[2],ls='-',label='Credit Card',fillstyle='none',linewidth = 4.0)
ax.plot(subprime_firsttoroll.loc[subprime_firsttoroll['date']>=start_date].xs(1,level="rollFM")['date'],subprime_firsttoroll.loc[subprime_firsttoroll['date']>=start_date].xs(1,level="rollFM")['firstroll_probab'], c=c[3],ls='-',label='Mortgage',fillstyle='none',linewidth = 4.0)

ax.set_xticklabels([str(i) for i in subprime_firsttoroll['date'].values], fontsize=15)
ax.set_yticklabels([str(i) for i in ax.get_yticks()],fontsize=15)
    
ax.tick_params(axis='x',colors='black')
ax.tick_params(axis='y', colors='black')
    
plt.xticks(rotation= 30)
plt.ylabel('%',fontsize = 16)
    
plt.legend(loc = "upper center", ncol = 4, prop = {'size':16}, bbox_to_anchor = (0.5,1.1), fancybox =True, frameon = False)
    
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(6)))
                               
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%y"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%b-%y"))
                                 
ax.grid(axis='y',linestyle='--',dashes=(5,5))
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
                                 
ax.yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_color('gray')

plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

fig1.suptitle('First to Roll into DQ 90+ when Other Categories Stay Current, Below 660 FICO',fontsize = 18)

plt.show()

In [ ]:
save_chart(fig1,"Subprime First to Roll")

### Nearprime Priority of Payment

In [ ]:
fig2=plt.figure(figsize=(12,6))
ax=fig2.add_subplot(1,1,1)

c=['#002060','#00BDF2','#99ABC7','#CCF2FC','99E2E3','#00B0B9','#FFD700','#A9561E','#E6DAA6']

ax.plot(nearprime_firsttoroll.loc[nearprime_firsttoroll['date']>=start_date].xs(1,level="rollAuto")['date'],nearprime_firsttoroll.loc[nearprime_firsttoroll['date']>=start_date].xs(1,level="rollAuto")['firstroll_probab'], c=c[0],ls='-',label='Auto',fillstyle='none',linewidth = 4.0)
ax.plot(nearprime_firsttoroll.loc[nearprime_firsttoroll['date']>=start_date].xs(1,level="rollPL")['date'],nearprime_firsttoroll.loc[nearprime_firsttoroll['date']>=start_date].xs(1,level="rollPL")['firstroll_probab'], c=c[1],ls='-',label='Personal Loan',fillstyle='none',linewidth = 4.0)
ax.plot(nearprime_firsttoroll.loc[nearprime_firsttoroll['date']>=start_date].xs(1,level="rollBC")['date'],nearprime_firsttoroll.loc[nearprime_firsttoroll['date']>=start_date].xs(1,level="rollBC")['firstroll_probab'], c=c[2],ls='-',label='Credit Card',fillstyle='none',linewidth = 4.0)
ax.plot(nearprime_firsttoroll.loc[nearprime_firsttoroll['date']>=start_date].xs(1,level="rollFM")['date'],nearprime_firsttoroll.loc[nearprime_firsttoroll['date']>=start_date].xs(1,level="rollFM")['firstroll_probab'], c=c[3],ls='-',label='Mortgage',fillstyle='none',linewidth = 4.0)

ax.set_xticklabels([str(i) for i in nearprime_firsttoroll['date'].values], fontsize=15)
ax.set_yticklabels([str(i) for i in ax.get_yticks()],fontsize=15)
    
ax.tick_params(axis='x',colors='black')
ax.tick_params(axis='y', colors='black')
    
plt.xticks(rotation= 30)
plt.ylabel('%',fontsize = 16)
    
plt.legend(loc = "upper center", ncol = 4, prop = {'size':16}, bbox_to_anchor = (0.5,1.1), fancybox =True, frameon = False)
    
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(6)))
                               
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%y"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%b-%y"))
                                 
ax.grid(axis='y',linestyle='--',dashes=(5,5))
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
                                 
ax.yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_color('gray')

plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

fig2.suptitle('First to Roll into DQ 90+ when Other Categories Stay Current, 660-720 FICO',fontsize = 18)

plt.show()

In [ ]:
save_chart(fig2,'Nearprime First to Roll')

### Prime Priority of Payment

In [ ]:
fig=plt.figure(figsize=(12,6))
ax=fig.add_subplot(1,1,1)

c=['#002060','#00BDF2','#99ABC7','#CCF2FC','99E2E3','#00B0B9','#FFD700','#A9561E','#E6DAA6']

ax.plot(prime_firsttoroll.loc[prime_firsttoroll['date']>=start_date].xs(1,level="rollAuto")['date'],prime_firsttoroll.loc[prime_firsttoroll['date']>=start_date].xs(1,level="rollAuto")['firstroll_probab'], c=c[0],ls='-',label='Auto',fillstyle='none',linewidth = 4.0)
ax.plot(prime_firsttoroll.loc[prime_firsttoroll['date']>=start_date].xs(1,level="rollPL")['date'],prime_firsttoroll.loc[prime_firsttoroll['date']>=start_date].xs(1,level="rollPL")['firstroll_probab'], c=c[1],ls='-',label='Personal Loan',fillstyle='none',linewidth = 4.0)
ax.plot(prime_firsttoroll.loc[prime_firsttoroll['date']>=start_date].xs(1,level="rollBC")['date'],prime_firsttoroll.loc[prime_firsttoroll['date']>=start_date].xs(1,level="rollBC")['firstroll_probab'], c=c[2],ls='-',label='Credit Card',fillstyle='none',linewidth = 4.0)
ax.plot(prime_firsttoroll.loc[prime_firsttoroll['date']>=start_date].xs(1,level="rollFM")['date'],prime_firsttoroll.loc[prime_firsttoroll['date']>=start_date].xs(1,level="rollFM")['firstroll_probab'], c=c[3],ls='-',label='Mortgage',fillstyle='none',linewidth = 4.0)

ax.set_xticklabels([str(i) for i in prime_firsttoroll['date'].values], fontsize=15)
ax.set_yticklabels([str(i) for i in ax.get_yticks()],fontsize=15)
    
ax.tick_params(axis='x',colors='black')
ax.tick_params(axis='y', colors='black')
    
plt.xticks(rotation= 30)
plt.ylabel('%',fontsize = 16)
    
plt.legend(loc = "upper center", ncol = 4, prop = {'size':16}, bbox_to_anchor = (0.5,1.1), fancybox =True, frameon = False)
    
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(6)))
                               
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%y"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%b-%y"))
                                 
ax.grid(axis='y',linestyle='--',dashes=(5,5))
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
                                 
ax.yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_color('gray')

plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

fig.suptitle('First to Roll into DQ 90+ when Other Categories Stay Current, above 720 FICO',fontsize = 18)

plt.show()

In [ ]:
save_chart(fig,'Prime First to Roll')

## -------------------------Seperator -----------------------

### Approach #2 - Last to Roll (Only Type to Remain Current)

### Fico Distribution for Borrowers that have 3 types of loan rolling into 90+ DQ within a 6 month rolling window

get the remaining one left type that has roll flag value of 0

In [ ]:
roll90_3cat_pivot = payment_priority_pivot.xs(3,level='rollProdTypes2').groupby(level=[0,'date','fico_bucket'])['unique_borrower_count'].sum()

#roll90_3cat_pivot=roll90_3cat_pivot.reset(level='date')

In [ ]:
subprime_lasttoremain = Get_Lasttoremain_probab(payment_priority_pivot,'below 660')
nearprime_lasttoremain = Get_Lasttoremain_probab(payment_priority_pivot,'660-720')
prime_lasttoremain = Get_Lasttoremain_probab(payment_priority_pivot,'above 720')

In [ ]:
subprime_lasttoremain['date']=pd.to_datetime(subprime_lasttoremain['date'], infer_datetime_format=True)
nearprime_lasttoremain['date']=pd.to_datetime(nearprime_lasttoremain['date'], infer_datetime_format=True)
prime_lasttoremain['date']=pd.to_datetime(prime_lasttoremain['date'], infer_datetime_format=True)

In [ ]:
subprime_lasttoremain.tail(10)

### Subprime Last to Roll

In [ ]:
fig=plt.figure(figsize=(12,6))
ax=fig.add_subplot(1,1,1)

c=['#002060','#00BDF2','#99ABC7','#CCF2FC','99E2E3','#00B0B9','#FFD700','#A9561E','#E6DAA6']

ax.plot(subprime_lasttoremain.loc[subprime_lasttoremain['date']>=start_date].xs(0,level="rollAuto")['date'],subprime_lasttoremain.loc[subprime_lasttoremain['date']>=start_date].xs(0,level="rollAuto")['Lastroll_probab'], c=c[0],ls='-',label='Auto',fillstyle='none',linewidth = 4.0)
ax.plot(subprime_lasttoremain.loc[subprime_lasttoremain['date']>=start_date].xs(0,level="rollPL")['date'],subprime_lasttoremain.loc[subprime_lasttoremain['date']>=start_date].xs(0,level="rollPL")['Lastroll_probab'], c=c[1],ls='-',label='Personal Loan',fillstyle='none',linewidth = 4.0)
ax.plot(subprime_lasttoremain.loc[subprime_lasttoremain['date']>=start_date].xs(0,level="rollBC")['date'],subprime_lasttoremain.loc[subprime_lasttoremain['date']>=start_date].xs(0,level="rollBC")['Lastroll_probab'], c=c[2],ls='-',label='Credit Card',fillstyle='none',linewidth = 4.0)
ax.plot(subprime_lasttoremain.loc[subprime_lasttoremain['date']>=start_date].xs(0,level="rollFM")['date'],subprime_lasttoremain.loc[subprime_lasttoremain['date']>=start_date].xs(0,level="rollFM")['Lastroll_probab'], c=c[3],ls='-',label='Mortgage',fillstyle='none',linewidth = 4.0)

ax.set_xticklabels([str(i) for i in subprime_lasttoremain['date'].values], fontsize=15)
ax.set_yticklabels([str(i) for i in ax.get_yticks()],fontsize=15)
    
ax.tick_params(axis='x',colors='black')
ax.tick_params(axis='y', colors='black')
    
plt.xticks(rotation= 30)
plt.ylabel('%',fontsize = 16)
    
plt.legend(loc = "upper center", ncol = 4, prop = {'size':16}, bbox_to_anchor = (0.5,1.1), fancybox =True, frameon = False)
    
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(6)))
                               
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%y"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%b-%y"))
                                 
ax.grid(axis='y',linestyle='--',dashes=(5,5))
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
                                 
ax.yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_color('gray')

plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

fig.suptitle('Only Type to Remain Current When Other Categories all Roll into 90+ DQ, Below 660 FICO',fontsize = 18)

plt.show()

In [ ]:
save_chart(fig,'Subprime Last to Remain')

### Nearprime Last to Roll

In [ ]:
fig=plt.figure(figsize=(12,6))
ax=fig.add_subplot(1,1,1)

c=['#002060','#00BDF2','#99ABC7','#CCF2FC','99E2E3','#00B0B9','#FFD700','#A9561E','#E6DAA6']

ax.plot(nearprime_lasttoremain.loc[nearprime_lasttoremain['date']>=start_date].xs(0,level="rollAuto")['date'],nearprime_lasttoremain.loc[nearprime_lasttoremain['date']>=start_date].xs(0,level="rollAuto")['Lastroll_probab'], c=c[0],ls='-',label='Auto',fillstyle='none',linewidth = 4.0)
ax.plot(nearprime_lasttoremain.loc[nearprime_lasttoremain['date']>=start_date].xs(0,level="rollPL")['date'],nearprime_lasttoremain.loc[nearprime_lasttoremain['date']>=start_date].xs(0,level="rollPL")['Lastroll_probab'], c=c[1],ls='-',label='Personal Loan',fillstyle='none',linewidth = 4.0)
ax.plot(nearprime_lasttoremain.loc[nearprime_lasttoremain['date']>=start_date].xs(0,level="rollBC")['date'],nearprime_lasttoremain.loc[nearprime_lasttoremain['date']>=start_date].xs(0,level="rollBC")['Lastroll_probab'], c=c[2],ls='-',label='Credit Card',fillstyle='none',linewidth = 4.0)
ax.plot(nearprime_lasttoremain.loc[nearprime_lasttoremain['date']>=start_date].xs(0,level="rollFM")['date'],nearprime_lasttoremain.loc[nearprime_lasttoremain['date']>=start_date].xs(0,level="rollFM")['Lastroll_probab'], c=c[3],ls='-',label='Mortgage',fillstyle='none',linewidth = 4.0)

ax.set_xticklabels([str(i) for i in nearprime_lasttoremain['date'].values], fontsize=15)
ax.set_yticklabels([str(i) for i in ax.get_yticks()],fontsize=15)
    
ax.tick_params(axis='x',colors='black')
ax.tick_params(axis='y', colors='black')
    
plt.xticks(rotation= 30)
plt.ylabel('%',fontsize = 16)
    
plt.legend(loc = "upper center", ncol = 4, prop = {'size':16}, bbox_to_anchor = (0.5,1.1), fancybox =True, frameon = False)
    
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(6)))
                               
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%y"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%b-%y"))
                                 
ax.grid(axis='y',linestyle='--',dashes=(5,5))
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
                                 
ax.yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_color('gray')

plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

fig.suptitle('Only Type to Remain Current When Other Categories all Roll into 90+ DQ, 660-720 FICO',fontsize = 18)

plt.show()

In [ ]:
save_chart(fig,'Nearprime Last to Remain')

### Prime Last to Roll

In [ ]:
fig=plt.figure(figsize=(12,6))
ax=fig.add_subplot(1,1,1)

c=['#002060','#00BDF2','#99ABC7','#CCF2FC','99E2E3','#00B0B9','#FFD700','#A9561E','#E6DAA6']

ax.plot(prime_lasttoremain.loc[prime_lasttoremain['date']>=start_date].xs(0,level="rollAuto")['date'],prime_lasttoremain.loc[prime_lasttoremain['date']>=start_date].xs(0,level="rollAuto")['Lastroll_probab'], c=c[0],ls='-',label='Auto',fillstyle='none',linewidth = 4.0)
ax.plot(prime_lasttoremain.loc[prime_lasttoremain['date']>=start_date].xs(0,level="rollPL")['date'],prime_lasttoremain.loc[prime_lasttoremain['date']>=start_date].xs(0,level="rollPL")['Lastroll_probab'], c=c[1],ls='-',label='Personal Loan',fillstyle='none',linewidth = 4.0)
ax.plot(prime_lasttoremain.loc[prime_lasttoremain['date']>=start_date].xs(0,level="rollBC")['date'],prime_lasttoremain.loc[prime_lasttoremain['date']>=start_date].xs(0,level="rollBC")['Lastroll_probab'], c=c[2],ls='-',label='Credit Card',fillstyle='none',linewidth = 4.0)
ax.plot(prime_lasttoremain.loc[prime_lasttoremain['date']>=start_date].xs(0,level="rollFM")['date'],prime_lasttoremain.loc[prime_lasttoremain['date']>=start_date].xs(0,level="rollFM")['Lastroll_probab'], c=c[3],ls='-',label='Mortgage',fillstyle='none',linewidth = 4.0)

ax.set_xticklabels([str(i) for i in prime_lasttoremain['date'].values], fontsize=15)
ax.set_yticklabels([str(i) for i in ax.get_yticks()],fontsize=15)
    
ax.tick_params(axis='x',colors='black')
ax.tick_params(axis='y', colors='black')
    
plt.xticks(rotation= 30)
plt.ylabel('%',fontsize = 16)
    
plt.legend(loc = "upper center", ncol = 4, prop = {'size':16}, bbox_to_anchor = (0.5,1.1), fancybox =True, frameon = False)
    
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(6)))
                               
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%y"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%b-%y"))
                                 
ax.grid(axis='y',linestyle='--',dashes=(5,5))
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
                                 
ax.yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_color('gray')

plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

fig.suptitle('Only Type to Remain Current When Other Categories all Roll into 90+ DQ, Above 720 FICO',fontsize = 18)

plt.show()

In [ ]:
save_chart(fig,'Prime Last to Remain')